In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from common import analyse_power, remove_outliers
from constants import T_s
from bmrs import read_generation_files
from datetime import date
from pathlib import Path

In [ ]:
projected_energy_TWh=200
projected_power_GW = projected_energy_TWh * 1000/24/365
projected_power_GW

In [ ]:
pathbase = '../data/cdrc/DomesticEnergyProviderDataset/DEP2015_SF_CDRC'

In [ ]:
with open(pathbase + '.natural_gas.npy', 'rb') as f:
    gas_power_raw = np.load(f)
with open(pathbase + '.electricity.npy', 'rb') as f:
    elec_power_raw = np.load(f)

In [ ]:
gas_power = np.nan_to_num(remove_outliers(gas_power_raw))
elec_power = np.nan_to_num(remove_outliers(elec_power_raw))
combined_power = gas_power/3 + elec_power
combined_power_mean = np.mean(combined_power)
combined_power_mean

In [ ]:
projected_consumption = (projected_power_GW/combined_power_mean) * combined_power
np.mean(projected_consumption)

In [ ]:
time = pd.date_range(start='2030-01-01', end='2030-12-31 23:30', periods=365*48)
consumer_ds = pd.Series(index=time,data=projected_consumption)
consumer_ds.plot(figsize=(12,8), 
             grid=True, 
             xlabel="Time", 
             ylabel="Power [GW]",
             title="Projected electricity consumption 2030")

In [ ]:
wind_power_raw = read_generation_files(Path("../data/bmrs/2019/FUELINST"))
wind_power = wind_power_raw.to_numpy()
wind_power_mean = np.mean(wind_power)
wind_power_mean

In [ ]:
projected_windpower = (projected_power_GW/wind_power_mean) * wind_power[:-1]
np.mean(projected_windpower)

In [ ]:
producer_power_ds = pd.Series(index=time,data=projected_windpower)
producer_power_ds.plot(figsize=(12,8), 
             grid=True, 
             xlabel="Time", 
             ylabel="Power [GW]",
             title="Projected electricity production 2030")

In [ ]:
power_difference = projected_windpower - projected_consumption

In [ ]:
balance_power_ds = pd.Series(index=time,data=power_difference)
balance_power_ds.plot(figsize=(12,8), 
             grid=True, 
             xlabel="Time", 
             ylabel="Power [GW]",
             title="Projected electricity production consumption difference 2030")

In [ ]:
result = analyse_power(power_difference)

In [ ]:
df = pd.Series(index=result.frequencies[1:]*3600*24*365,
                  data=result.spectrum[1:])
df.plot(figsize=(12,8), 
        grid=True, 
        color='r', 
        logx=True, 
        xlabel="[cycles/year]", 
        ylabel="Cycle amplitude [GW]",
        title=f"Frequency spectrum of electricity power for 2015"
       )

In [ ]:
result.print()

In [ ]:
for index, value in zip(result.peak_indices, result.peak_values):
    energy = value/np.pi/result.frequencies[index]/3600
    print(f"\tEnergy @ bin {index} = {energy:.3f}GWh")
    batteries = energy*1000*1000/100
    print(f"\Total Batteries @ bin {index} = {batteries:.3f} batteries")
    print(f"\Home Batteries @ bin {index} = {batteries/30000000:.3f} batteries")


In [ ]:
energy_balance = np.cumsum(power_difference) * T_s / 3600/1000
energy_store = energy_balance - np.min(energy_balance)

In [ ]:
energy_store_ds = pd.Series(index=time,data=energy_store)
energy_store_ds.plot(figsize=(12,8), 
             grid=True, 
             xlabel="Time", 
             ylabel="Energy stored [TWh]",
             title="Projected energy store evolution in 2030")

In [ ]:
capacity = np.max(energy_store) - np.min(energy_store)
capacity

In [ ]:
tesla_batteries = capacity*1000*1000*1000/100
tesla_batteries

In [ ]:
tesla_batteries/30000000

In [ ]:
result = analyse_power(energy_store)

In [ ]:
df = pd.Series(index=result.frequencies[1:]*3600*24*365,
                  data=result.spectrum[1:])
df.plot(figsize=(12,8), 
        grid=True, 
        color='r', 
        logx=True, 
        xlabel="[cycles/year]", 
        ylabel="Cycle amplitude [TWh]",
        title=f"Frequency spectrum of electricity store for 2030"
       )

In [ ]:
result.print()

In [ ]:
22.831050228310502/1.21

In [ ]:
capacity